# Project 02 - Image Processing

## Student Information

- Full name: Nguyen Minh Nhat
- Student ID: 22127309
- Class: 22CLC05

## Required Libraries

In [1]:
# IMPORT YOUR LIBS HERE
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

## Function Definitions

In [2]:
def read_img(img_path) -> np.ndarray:
    '''
    Read image from img_path

    Parameters
    ----------
    img_path : str
        Path of image

    Returns
    -------
        Image
    '''
    return np.array(Image.open(img_path))
    


def show_img(img) -> None:
    '''
    Show image

    Parameters
    ----------
    img : <your type>
        Image
    '''
    plt.imshow(img)




def save_img(img, img_path):
    '''
    Save image to img_path

    Parameters
    ----------
    img : <your type>
        Image
    img_path : str
        Path of image
    '''
    Image.fromarray(img).save(img_path)


#--------------------------------------------------------------------------------
# MY FUNCTIONS

def pad_image(img, target_shape):
    '''
    Pad the image to the target shape with zeros (black padding).

    Parameters
    ----------
    img : np.ndarray
        Image to pad
    target_shape : tuple
        Target shape (height, width, channels)
    
    Returns
    -------
    padded_img : np.ndarray
        Padded image
    '''
    padded_img = np.full(target_shape, 255, dtype=img.dtype) 
    #np.zeros(target_shape, dtype=img.dtype) 

    padded_img[:img.shape[0], :img.shape[1], ...] = img
    return padded_img

def show_img_compare(img1: np.ndarray, img2: np.ndarray, 
                     title1: str = 'Original', title2: str = 'Modified', 
                     axis: bool = False, space: int = 10) -> None:
    '''
    Show two images side by side, even if they are of different sizes, with space between them.

    Parameters
    ----------
    img1 : np.ndarray
        Image 1
    img2 : np.ndarray
        Image 2
    title1 : str
        Title of image 1
    title2 : str
        Title of image 2
    axis : bool
        Show axis or not
    space : int
        Space in pixels between the images
    '''

    target_shape = (
        max(img1.shape[0], img2.shape[0]), 
        max(img1.shape[1], img2.shape[1]), 
        img1.shape[2] if len(img1.shape) > 2 else 1
    )
    img1_padded = pad_image(img1, target_shape)
    img2_padded = pad_image(img2, target_shape)
    spacer = np.full((target_shape[0], space, target_shape[2]), 255, dtype=np.uint8)
    combined_img = np.concatenate((img1_padded, spacer, img2_padded), axis=1)
    plt.imshow(combined_img)
    #plt.title(f'{title1} (Left) | {title2} (Right)')
    if not axis:
        plt.axis('off')
    plt.show()

def adjust_brightness(img: np.ndarray, factor: float) -> np.ndarray:
    """
    Adjust the brightness of an image by multiplying the RGB values by a factor.

    Parameters
    ----------
    img : np.ndarray
        Image
    factor : float
        Factor to adjust the brightness. A factor greater than 0 will increase the brightness
        and a factor less than 0 will decrease the brightness.

    Returns
    -------
    np.ndarray
        Image with adjusted brightness
    """
    return np.clip(img.astype(np.float32) + factor, 0, 255).astype(np.uint8)
    
def adjust_contrast(img: np.ndarray, factor: float) -> np.ndarray:
    """
    Adjust the contrast of an image by subtracting the mean RGB value and adding a factor times the standard deviation.

    Parameters
    ----------
    img : np.ndarray
        Image
    factor : float
        Factor to adjust the contrast. A factor greater than 1 will increase the contrast
        and a factor less than 1 will decrease the contrast.

    Returns
    -------
    np.ndarray
        Image with adjusted contrast
    """
    return np.clip(img.astype(np.float32) * factor, 0, 255).astype(np.uint8)




def flip_image(img: np.ndarray, is_horizontal: bool = True) -> np.ndarray:
    """
    Flip image horizontally or vertically

    Parameters
    ----------
    img : np.ndarray
        Image
    is_horizontal : bool, optional
        If True, flip horizontally. Otherwise, flip vertically.
        Default is True.

    Returns
    -------
    np.ndarray
        Flipped image
    """
    if is_horizontal:
        return img[:, ::-1]
    else:
        return img[::-1, :]

def apply_color_transform(image : np.ndarray, weights : np.ndarray) -> np.ndarray:
    """
    Apply a color transformation to an image using the provided weights for the RGB channels.

    Parameters:
    ----------
    image (numpy.ndarray): Input image with shape (height, width, 3).
    weights (numpy.ndarray): Weights for the RGB channels with shape (3,3).

    Returns:
    -------
    numpy.ndarray: Transformed image with shape (height, width, 3).
    """
    return np.clip(np.einsum('ijk,kl->ijl', image, weights.T), 0, 255).astype(np.uint8)
 


def rgb2grey(img) -> np.ndarray:
    '''
    Convert RGB image to greyscale image

    Parameters
    ----------
    img : np.ndarray
        RGB image

    Returns
    -------
    np.ndarray
        greyscale image
    '''
    weights = np.array([[0.2125, 0.7154, 0.0721], 
                        [0.2125, 0.7154, 0.0721], 
                        [0.2125, 0.7154, 0.0721]])
    return apply_color_transform(img, weights)

def rgb2sepia(img) :
    '''
    Convert RGB image to sepia image

    Parameters
    ----------
    img : np.ndarray
        RGB image

    Returns
    -------
    np.ndarray
        Sepia image
    '''
    weights = np.array([[.393, .769, .189],
                         [.349, .686, .168],
                         [.272, .534, .131]])
    return apply_color_transform(img, weights)
    

def convolution_2d(image, kernel):
    """
    Perform 2D convolution on an image using a kernel.

    Parameters:
    ----------
    image (numpy.ndarray): Input image with shape (height, width, depth).
    kernel (numpy.ndarray): Convolution kernel with shape (kernel_height, kernel_width).

    Returns:
    -------
    numpy.ndarray: Resulting image with shape (height, width
    """
    height, width, depth = image.shape
    w_h, w_w = kernel.shape
    array = np.pad(image, ((w_h // 2, w_h // 2), (w_w // 2, w_w // 2), (0, 0)), mode='edge').astype(np.float32)
    kernel = kernel.flatten()
    result = np.zeros((height, width, depth), dtype=np.float32)
    for c in range(depth):
        windows = np.lib.stride_tricks.sliding_window_view(array[:, :, c], (w_h, w_w)).reshape(height, width, w_h *w_w)
        result[:, :, c] = np.einsum('ijk, k -> ij', windows, kernel)
    return result.clip(0, 255).astype(np.uint8)


def apply_gaussian_blur_5x5(image):
    """
    Apply a 5x5 Gaussian blur to an image.

    Parameters:
    ----------
    image (numpy.ndarray): Input image with shape (height, width, depth).

    Returns:
    -------
    numpy.ndarray: Resulting image with shape (height, width, depth).
    """
    kernel = np.array([[1, 4, 6, 4, 1],
                       [4, 16, 24, 16, 4],
                       [6, 24, 36, 24, 6],
                       [4, 16, 24, 16, 4],
                       [1, 4, 6, 4, 1]]) / 256
    return convolution_2d(image, kernel)

def apply_sharpen_filter(image):
    """
    Apply a sharpen filter to an image.

    Parameters:
    ----------
    image (numpy.ndarray): Input image with shape (height, width, depth).

    Returns:
    -------
    numpy.ndarray: Resulting image with shape (height, width, depth).
    """
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    return convolution_2d(image, kernel)

def crop_center(image: np.ndarray, percentage: float) -> np.ndarray:
    """
    Crop an image from the center.

    Parameters:
    ----------
    image (numpy.ndarray): Input image with shape (height, width, 3).
    percentage (float): Percentage of the image to keep. This value should be in the range (0, 1].

    Returns:
    -------
    numpy.ndarray: Cropped image with shape (height, width, 3).

    Note:
    -----
    - The percentage value should be between 0 and 1 (inclusive of 1, exclusive of 0).
    - If the percentage is less than or equal to 0, it will be clipped to 0.001.
    - If the percentage is greater than 1, it will be clipped to 1.
    """
    if percentage <= 0 or percentage > 1:
        percentage = np.clip(percentage, 1e-3, 1)
        print(f"The percentage value should be in the range (0, 1]. It has been clipped to {percentage} for this operation.")
    
    factor = (1 / percentage) ** 0.5
    h, w = image.shape[:2]
    h_new, w_new = int(h / factor), int(w / factor)
    h_start, w_start = (h - h_new) // 2, (w - w_new) // 2
    return image[h_start:h_start + h_new, w_start:w_start + w_new]

def circular_crop(image: np.ndarray) -> np.ndarray:
    """
    Crop a 2D image to a circular region centered in the image.

    Parameters
    ----------
    image : np.ndarray
        Input 2D numpy array (height, width).

    Returns
    -------
    np.ndarray
        Image with a circular region preserved; non-circular areas set to zero.

    Notes
    -----
    The circle is centered in the image and has the radius of half the smaller dimension of the image.
    """
    center = np.array([image.shape[0] + 1, image.shape[1] + 1]) / 2
    square_radius = np.square(min(np.floor(center)))
    grid = np.indices((image.shape[0], image.shape[1]))
    diff = grid - center[:, np.newaxis, np.newaxis]
    mask = np.einsum('ijk, ijk->jk', diff, diff) <= square_radius
    return image * mask[:, :, np.newaxis]


def ellipses_crop(image: np.ndarray) -> np.ndarray:
    """
    Crop an image to an ellipse shape.

    Parameters:
    ----------
    image (numpy.ndarray): Input image with shape (height, width, 3).

    Returns:
    -------
    numpy.ndarray: Cropped image with shape (height, width, 3).
    Note:
    -----
    - The image should be square because the axes are calculated based on the height and width of the image.
    """
    center = np.array([image.shape[0] + 1, image.shape[1] + 1]) / 2
    grid = np.indices((image.shape[0], image.shape[1]))
    axes = np.array([image.shape[0] * 0.6, image.shape[1] * 0.365])
    rf = 2**(-0.5) # rotation factor = sin(45) = -sin(-45) = cos(45) = cos(-45)
    diff = grid - center[:, np.newaxis, np.newaxis]
    #factor 1 (rf, rf) (-rf, rf)
    first_a_rot = np.einsum('ijk, i->jk', diff, np.array([rf, rf]) / axes[0]) ** 2
    first_b_rot = np.einsum('ijk, i->jk', diff, np.array([-rf, rf]) / axes[1]) ** 2
    #factor 2 (-rf, rf) (-rf, -rf)
    second_a_rot = np.einsum('ijk, i->jk', diff, np.array([-rf, rf]) / axes[0]) ** 2
    second_b_rot = np.einsum('ijk, i->jk', diff, np.array([-rf, -rf]) / axes[1]) ** 2
    mask = ((first_a_rot + first_b_rot) <= 1) | ((second_a_rot + second_b_rot) <= 1)
    return image * mask[:, :, np.newaxis]

def resize(image: np.ndarray, new_height: int, new_width: int) -> np.ndarray:
    """
    Resize an image to a new height and width using bilinear interpolation.

    Parameters
    ----------
    image : np.ndarray
        Input image as a 3D NumPy array with shape (height, width, depth).
    new_height : int
        Desired height of the resized image.
    new_width : int
        Desired width of the resized image.

    Returns
    -------
    np.ndarray
        Resized image as a 3D NumPy array with shape (new_height, new_width, depth).

    Notes
    -----
    The function uses bilinear interpolation to calculate the values of the resized image.
    It computes the new pixel values by considering the four neighboring pixels from the original image
    and their distances to the target pixel.
    """
    
    old_height, old_width, depth = image.shape
    scale = np.array([old_height / new_height, old_width / new_width]) 
    resized_image = np.zeros((new_height, new_width, depth), dtype=np.float32)
    y, x = np.indices((new_height, new_width), dtype=np.uint32) * scale[:, np.newaxis, np.newaxis]
    x = x.flatten()
    y = y.flatten()
    x1 = np.floor(x).astype(np.uint32)
    y1 = np.floor(y).astype(np.uint32)
    x2 = np.minimum(x1 + 1, old_width - 1)
    y2 = np.minimum(y1 + 1, old_height - 1)

    for i in range(depth):
        Ia = image[y1, x1, i]
        Ib = image[y2, x1, i]
        Ic = image[y1, x2, i]
        Id = image[y2, x2, i]
        
        wa = (x2 - x) * (y2 - y)
        wb = (x2 - x) * (y - y1)
        wc = (x - x1) * (y2 - y)
        wd = (x - x1) * (y - y1)

        resized_image[..., i] = (Ia * wa + Ib * wb + Ic * wc + Id * wd).reshape(new_height, new_width)
    
    return resized_image.astype(np.uint8)

<ins>Note:</ins> For clarity, include docstrings with each function.

## Your tests

In [3]:
#image = read_img('lena.png')
#save_img(adjust_brightness(image, 30), 'brightness.png')
#save_img(adjust_contrast(image, 1.5), 'contrast.png')
#save_img(flip_image(image, is_horizontal=True), 'flip_horizontal.png')
##save_img(flip_image(image, is_horizontal=False), 'flip_vertical.png')
#save_img(rgb2grey(image), 'rgb_to_grayscale.png')
#save_img(rgb2sepia(image), 'rgb_to_sepia.png')
#save_img(apply_gaussian_blur_5x5(image), 'blur.png')
#save_img(apply_sharpen_filter(image), 'sharpen.png')
#save_img(pad_image(crop_center(image, 0.5), image.shape), 'crop.png')
#save_img(circular_crop(image), 'circular_crop.png')
#save_img(ellipses_crop(image), 'elliptical_crop.png')
#save_img(resize(image, 1024, 1024), 'zoom_out.png')
#save_img(resize(image, 256, 256), 'zoom_in.png')

## Main FUNCTION

In [4]:
def main() -> None:
    """ Main function to run the program """
    choice = 13
    image_path = ""
    output_ext = ""
    while (True):
        try:
            if (choice == 13):
                print("Enter the path of the image you want to process (empty to exit): ", flush=True)
                image_path = input()
                if (image_path == ""):
                    break
                print("Enter the extension of the output image (e.g. jpg, png): ", flush=True)
                output_ext = input()
            image = read_img(image_path)
            print("0. Do everything, number and factor will be asigned by the program", flush=True)
            print("1. Change Brightness", flush=True)
            print("2. Change Contrast", flush=True)
            print("3. Flip Horizontally", flush=True)
            print("4. Flip Vertically", flush=True)
            print("5. Convert to Grayscale", flush=True)
            print("6. Convert to Sepia", flush=True)
            print("7. Apply Gaussian Blur", flush=True)
            print("8. Apply Sharpen Filter", flush=True)
            print("9. Crop Image", flush=True)
            print("10. Circular Crop", flush=True)
            print("11. Elliptical Crop", flush=True)
            print("12. Resize Image", flush=True)
            print("13. Enter new image", flush=True)
            print("14. Exit", flush=True)
            choice = int(input())
            if (choice == 0):
                new_image = adjust_brightness(image, 30)
                show_img_compare(image, new_image, 'Original', 'Brightness Changed')
                save_img(new_image, f'output_brightness.{output_ext}')
                new_image = adjust_contrast(image, 1.5)
                show_img_compare(image, new_image, 'Original', 'Contrast Changed')
                save_img(new_image, f'output_contrast.{output_ext}')
                new_image = flip_image(image, is_horizontal=True)
                show_img_compare(image, new_image, 'Original', 'Flipped Horizontally')
                save_img(new_image, f'output_flip_horizontal.{output_ext}')
                new_image = flip_image(image, is_horizontal=False)
                show_img_compare(image, new_image, 'Original', 'Flipped Vertically')
                save_img(new_image, f'output_flip_vertical.{output_ext}')
                new_image = rgb2grey(image)
                show_img_compare(image, new_image, 'Original', 'Grayscale')
                save_img(new_image, f'output_grayscale.{output_ext}')
                new_image = rgb2sepia(image)
                show_img_compare(image, new_image, 'Original', 'Sepia')
                save_img(new_image, f'output_sepia.{output_ext}')
                new_image = apply_gaussian_blur_5x5(image)
                show_img_compare(image, new_image, 'Original', 'Blurred')
                save_img(new_image, f'output_blur.{output_ext}')
                new_image = apply_sharpen_filter(image)
                show_img_compare(image, new_image, 'Original', 'Sharpened')
                save_img(new_image, f'output_sharpen.{output_ext}')
                new_image = crop_center(image, 0.5)
                show_img_compare(image, new_image, 'Original', 'Cropped')
                save_img(new_image, f'output_crop.{output_ext}')
                new_image = circular_crop(image)
                show_img_compare(image, new_image, 'Original', 'Circular Cropped')
                save_img(new_image, f'output_circular_crop.{output_ext}')
                new_image = ellipses_crop(image)
                show_img_compare(image, new_image, 'Original', 'Elliptical Cropped')
                save_img(new_image, f'output_elliptical_crop.{output_ext}')
                new_image = resize(image, 1024, 1024)
                show_img_compare(image, new_image, 'Original', 'Resized')
                save_img(new_image, f'output_resize.{output_ext}')
            elif (choice == 1):
                print("Enter the factor by which you want to change the brightness(positive for increase, negative for decrease): ", flush=True)
                factor = float(input())
                new_image = adjust_brightness(image, factor)
                show_img_compare(image, new_image, 'Original', 'Brightness Changed')
                save_img(new_image, f'output_brightness.{output_ext}')
            elif (choice == 2):
                print("Enter the factor by which you want to change the contrast: ", flush=True)
                factor = float(input())
                new_image = adjust_contrast(image, factor)
                show_img_compare(image, new_image, 'Original', 'Contrast Changed')
                save_img(new_image, f'output_contrast.{output_ext}')
            elif (choice == 3):
                new_image = flip_image(image, is_horizontal=True)
                show_img_compare(image, new_image, 'Original', 'Flipped Horizontally')
                save_img(new_image, f'output_flip_horizontal.{output_ext}')
            elif (choice == 4):
                new_image = flip_image(image, is_horizontal=False)
                show_img_compare(image, new_image, 'Original', 'Flipped Vertically')
                save_img(new_image, f'output_flip_vertical.{output_ext}')
            elif (choice == 5):
                new_image = rgb2grey(image)
                show_img_compare(image, new_image, 'Original', 'Grayscale')
                save_img(new_image, f'output_grayscale.{output_ext}')
            elif (choice == 6):
                new_image = rgb2sepia(image)
                show_img_compare(image, new_image, 'Original', 'Sepia')
                save_img(new_image, f'output_sepia.{output_ext}')
            elif (choice == 7):
                new_image = apply_gaussian_blur_5x5(image)
                show_img_compare(image, new_image, 'Original', 'Blurred')
                save_img(new_image, f'output_blur.{output_ext}')
            elif (choice == 8):
                new_image = apply_sharpen_filter(image)
                show_img_compare(image, new_image, 'Original', 'Sharpened')
                save_img(new_image, f'output_sharpen.{output_ext}')
            elif (choice == 9):
                percentage = float(input("Enter the percentage by which you want to crop the image in  (0.0, 1.0]: "))
                new_image = crop_center(image, percentage)
                show_img_compare(image, new_image, 'Original', 'Cropped')
                save_img(new_image, f'output_crop.{output_ext}')
            elif (choice == 10):
                new_image = circular_crop(image)
                show_img_compare(image, new_image, 'Original', 'Circular Cropped')
                save_img(new_image, f'output_circular_crop.{output_ext}')
            elif (choice == 11):
                new_image = ellipses_crop(image)
                show_img_compare(image, new_image, 'Original', 'Elliptical Cropped')
                save_img(new_image, f'output_elliptical_crop.{output_ext}')
            elif (choice == 12):
                print("Enter the new height of the image: ", flush=True)
                new_height = int(input())
                print("Enter the new width of the image: ", flush=True)
                new_width = int(input())
                new_image = resize(image, new_height, new_width)
                show_img_compare(image, new_image, 'Original', 'Resized')
                save_img(new_image, f'output_resize.{output_ext}')
            elif (choice == 13):
                continue
            elif (choice == 14):
                break
            else:
                print("Invalid choice. Please try again.")
                choice = 13
        except Exception as e:
            print(f"An error occurred: {e}")
            choice = 13

In [ ]:
if __name__ == "__main__":
    main()